In [2]:
# check gpu connection
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

C:\Users\JimHentai\anaconda3\envs\deep-learning\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\JimHentai\anaconda3\envs\deep-learning\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\JimHentai\anaconda3\envs\deep-learning\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\JimHentai\anaconda3\envs\dee

SystemError: GPU device not found

In [ ]:
# connect to drive

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
pip install pdfplumber

     |████████████████████████████████| 46 kB 2.0 MB/s 
     |████████████████████████████████| 5.6 MB 9.3 MB/s 
     |████████████████████████████████| 4.3 MB 24.2 MB/s 
     |████████████████████████████████| 139 kB 48.6 MB/s 
     |████████████████████████████████| 3.6 MB 33.0 MB/s 
  Created wheel for pdfplumber: filename=pdfplumber-0.6.0-py3-none-any.whl size=33688 sha256=bcb14cf893236fc972506ae42f038ddd6f0240ffa3f7b3d0e31bcacdfa1b8450
  Stored in directory: /root/.cache/pip/wheels/58/56/fe/2e93d842ffa9ea97746c1ab253d43502ed61c0689361a0224e
Successfully built pdfplumber
  Attempting uninstall: Pillow
    Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Successfully uninstalled Pillow-7.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is inc

In [ ]:
import os
import pdfplumber
import pandas as pd
import time
import numpy as np
import re

In [ ]:
# This block converts pdf's into pandas dataframe as a string
start = time.time()

def get_filepaths(directory):
  
    """
    This function will generate the file names in a directory 
    tree by walking the tree either top-down or bottom-up. For each 
    directory in the tree rooted at directory top (including top itself), 
    it yields a 3-tuple (dirpath, dirnames, filenames).
    """
    file_paths = []  # List which will store all of the full filepaths.

    # Walk the tree.
    for root, directories, files in os.walk(directory):
        for filename in files:
            # Join the two strings in order to form the full filepath.
            filepath = os.path.join(root, filename)
            file_paths.append(filepath)  # Add it to the list.

    return file_paths  # Self-explanatory.

# run with gpu
with tf.device('/device:GPU:0'):  

  # court that we want to convert to csv
  court_nums = [1,2,3,4,5,6,7,8,9,10,11,12]

  files = get_filepaths("/content/gdrive/MyDrive/CS_490_DATA/Court_"+str(court_num))


  # randomly select a subset of files
  files = np.random.choice(files, 1000, replace= False)

  # Find case_id
  def find_case_id(file_index):
    name = (files[count].split('/')[6])
    case_id_temp = name.split('-')
    case_id = case_id_temp[2] + "-" + case_id_temp[3]
    case_problem_identifier = case_id_temp[4][0]

    return case_id,case_problem_identifier

  # Loop through all pages in pdf file
  # **ADDING Extracting the judges name from the first page
  def loop(file):
    with tf.device('/device:GPU:0'):  
      text = ""
      for page in range(len(file.pages)):
              text += file.pages[page].extract_text()

      #For court 11 only
      #regDOTALL = re.compile(r'\bBefore\b.*\bJudges\b\.', re.DOTALL)
      #judge_line = regDOTALL.findall(text)
      #judges_list = re.findall(r'(\b[A-Z]+\b\s\b[A-Z]+\b|\b[A-Z]+\b)', judge_line[0])
      # Not sure how we want to store judges, each in own column?, as a list?,
      # just as another string with all of them concatenated?

      return text    #, judges_list - took out

  # Make new function for judges?
  # For court 11, format is: Before ED CARNES, Chief Judge, MARTIN, and ROSENBAUM, Circuit Judges.
  # So usually: Before JUDGE_FIRST JUDGE_LAST, Chief Judge, JUDGE2_LAST, and JUDGE3_LAST, Circuit Judges.
  # Always starts with "Before", sometimes no "and", names always in caps, always separated
  # by commas, and typically ends with "Judge(s)."
  

  data = pd.DataFrame(columns=['Court_id','Case_id',"Opinion"])   #Added judges here - took out

  count = 0
  completed_count = 0

  # loop through all files in directory
  for file in files:
    if completed_count == 500:
        break;
    with tf.device('/device:GPU:0'):  
      with pdfplumber.open(file) as f:
          case_id, case_problem_id = find_case_id(count)
          if len(f.pages) < 2 or case_problem_id == 1:
            count+=1
            print(count)
            continue
          all_text = loop(f)
          data.loc[len(data.index)] = [court_num, case_id ,all_text]  # Added judges column - took out
      completed_count +=1
      count+=1
      # keep track of where we are
      print(count)
      data.to_csv("/content/gdrive/MyDrive/CS_490_DATA/Court_"+str(court_num)+".csv")
  
  # Convert data to csv
  data.to_csv("/content/gdrive/MyDrive/CS_490_DATA/Court_"+str(court_num)+".csv")


  end = time.time()
  print(end-start)

KeyboardInterrupt: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Code to get number of pages/files

from pdfplumber import pdf
def get_filepaths(directory):
  with tf.device('/device:GPU:0'):
    file_paths = []  # List which will store all of the full filepaths.
    # Walk the tree.
    for root, directories, files in os.walk(directory):
        for filename in files:
            # Join the two strings in order to form the full filepath.
            filepath = os.path.join(root, filename)
            file_paths.append(filepath)  # Add it to the list.

    return file_paths  # Self-explanatory.

def number_of_pages(file):
  with tf.device('/device:GPU:0'):
    with pdfplumber.open(file) as f:
        return len(f.pages)

# list all court that you want included
court_num = [2]

with tf.device('/device:GPU:0'):
  for num in court_num:
    sum = 0
    lenOne_pages = 0
    file_paths = get_filepaths("/content/gdrive/MyDrive/CS_490_DATA/Court_"+str(num))
    for file in file_paths:
      num_pages = number_of_pages(file)
      if num_pages == 1:
        lenOne_pages += 1
      sum+= num_pages
    print("COURT_"+str(num) + ":" + str(len(file_paths)) + " Total Opionions")
    print("COURT_"+str(num) + ":" + str(sum) + " Pages")
    print("COURT_"+str(num) + ":" + str(lenOne_pages) + " Pages")

In [ ]:
import pandas as pd
court_nums = [1,2,3,4,5,6,7,8,9,10,11,12]
for courts in court_nums:
  data = pd.read_csv("/content/gdrive/MyDrive/CS_490_DATA/Court_"+str(courts)+".csv")
  # data = data.sample(n=500)
  # data.to_csv("/content/gdrive/MyDrive/CS_490_DATA/Court_"+str(courts)+".csv")
  print(len(data))



In [ ]:
import pandas as pd
import re

data = pd.read_csv("/content/gdrive/MyDrive/CS_490_DATA/Court_8.csv")

text = " ".join(review for review in data["Opinion"])

text = re.sub(r'[^\w\s]', '', text)

import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger') 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
from nltk.corpus import stopwords
from nltk import word_tokenize

with tf.device('/device:GPU:0'):  
  text_tokens = word_tokenize(text)
  tokens_without_sw = [word for word in text_tokens if not word in stopwords.words("english")]
  

In [ ]:
from collections import Counter
  
  
# split() returns list of all the words in the string
  
# Pass the split_it list to instance of Counter class.
Counter = Counter(tokens_without_sw)
  
# most_common() produces k frequently encountered
# input values and their respective counts.
most_occur = Counter.most_common(1000)
  
print(most_occur)

[('v', 4498), ('court', 3312), ('The', 3236), ('States', 3172), ('United', 3167), ('F3d', 2644), ('district', 2625), ('Filed', 2604), ('Appellate', 2379), ('Case', 2367), ('Cir', 2356), ('Date', 2249), ('Page', 2233), ('Entry', 2231), ('ID', 2231), ('8th', 2095), ('District', 1717), ('See', 1619), ('Court', 1543), ('____________', 1483), ('Mr', 1345), ('2', 1221), ('evidence', 1186), ('Circuit', 1174), ('In', 1038), ('We', 1035), ('judgment', 1028), ('also', 1027), ('US', 1013), ('sentence', 978), ('appeal', 976), ('would', 929), ('___________________________', 919), ('1', 888), ('review', 864), ('claim', 816), ('law', 805), ('courts', 803), ('No', 792), ('case', 768), ('USC', 748), ('must', 732), ('et', 726), ('al', 709), ('whether', 700), ('Id', 695), ('Eighth', 687), ('Missouri', 680), ('may', 664), ('For', 662), ('3', 662), ('motion', 648), ('A', 634), ('2020', 628), ('opinion', 621), ('Appeals', 608), ('Judge', 605), ('claims', 601), ('see', 600), ('record', 592), ('within', 572),

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

A = ['very good day', 'a random thought', 'maybe like this']
B = ['so fast and slow', 'the meaning of this', 'here you go']
C = [1, 2, 3]

pdt_items = pd.DataFrame({'A':A,'B':B,'C':C})

cv = CountVectorizer()

# use pd.DataFrame here to avoid your error and add your column name    
sample = pd.DataFrame(pdt_items['A']+','+pdt_items['B']+','+pdt_items['C'].astype('str'), columns=['Output'])

vectorized = cv.fit_transform(sample['Output'])

sample

,Output
0,"very good day,so fast and slow,1"
1,"a random thought,the meaning of this,2"
2,"maybe like this,here you go,3"
